<a href="https://colab.research.google.com/github/t-nakatani/signate_stu22/blob/exp/signate_stu_v5_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[KaggleタイタニックのNameだけで予測精度80%超えた話(BERT/TF2.0)](https://qiita.com/pocokhc/items/56273f40f57679f25341)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from google.colab import drive
drive.mount('/content/drive')
ROOT = "/content/drive/MyDrive/Colab Notebooks/signate/"

#学習用データと評価用データの読み込み
train = pd.read_csv(os.path.join(ROOT, "train_cleaned.csv"))
test = pd.read_csv(os.path.join(ROOT, "test_cleaned.csv"))

jobdic = dict(zip([1, 2, 3, 4], ['DataScientist', 'ML Engineer', 'Software Engineer', 'Consultant']))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -q transformers==3

In [3]:
import numpy as np
import transformers
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizerFast, RobertaModel
from torch import optim
from torch import cuda
import time
from matplotlib import pyplot as plt
from tqdm import tqdm

In [4]:
# Datasetの定義
class CreateDataset(Dataset):
  def __init__(self, X, y, tokenizer, max_len):
    self.X = X
    self.y = y
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):  # len(Dataset)で返す値を指定
    return len(self.X)

  def __getitem__(self, index):  # Dataset[index]で返す値を指定
    text = self.X[index]
    inputs = self.tokenizer.encode_plus(
      text,
      add_special_tokens=True,
      max_length=self.max_len,
      pad_to_max_length=True,
      truncation=True
    )
    ids = inputs['input_ids']
    mask = inputs['attention_mask']
    labels = self.y[index]

    return {
      'ids': torch.LongTensor(ids),
      'mask': torch.LongTensor(mask),
      'labels': torch.Tensor(labels)
    }

In [5]:
# BERT分類モデルの定義
class BERTClass(torch.nn.Module):
  def __init__(self, drop_rate, otuput_size):
    super().__init__()
    self.bert = RobertaModel.from_pretrained('roberta-base')
    self.drop = torch.nn.Dropout(drop_rate)
    self.fc = torch.nn.Linear(768, otuput_size)  # BERTの出力に合わせて768次元を指定

  def forward(self, ids, mask):
    _, out = self.bert(ids, attention_mask=mask)
    out = self.fc(self.drop(out))
    return out

In [6]:
def calculate_loss_and_accuracy(model, criterion, loader, device, epoch, max_epoch):
  """ 損失・正解率を計算"""
  model.eval()
  loss = 0.0
  total = 0
  correct = 0
  with torch.no_grad():
    with tqdm(loader) as pbar:
      pbar.set_description('valid')
      for i, data in enumerate(pbar):
        ids = data['ids'].to(device)
        mask = data['mask'].to(device)
        labels = data['labels'].to(device)


        outputs = model(ids, mask)
        loss += criterion(outputs, labels).item()

        pred = torch.argmax(outputs, dim=-1).cpu().numpy() # バッチサイズの長さの予測ラベル配列
        labels = torch.argmax(labels, dim=-1).cpu().numpy()  # バッチサイズの長さの正解ラベル配列
        total += len(labels)
        correct += (pred == labels).sum().item()
        pbar.set_postfix(Loss=loss/(i+1), Accuracy=correct/total)

  return loss / len(loader), correct / total

In [7]:
def train_val(train_loader, val_loader, model_path):
  device = 'cuda' if cuda.is_available() else 'cpu'
  model = BERTClass(DROP_RATE, OUTPUT_SIZE).to(device)
  criterion = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)
  
  log_valid = []
  best_acc = correct = total = 0
  

  # train
  for epoch in range(NUM_EPOCHS):
    print('\nepoch:', epoch)
    loss_train = 0
    model.train()
    with tqdm(train_loader) as pbar:
      pbar.set_description('train')
      for i, data in enumerate(pbar):
        ids = data['ids'].to(device)
        mask = data['mask'].to(device)
        labels = data['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(ids, mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        loss_train += loss.item()
        pred = torch.argmax(outputs, dim=-1).cpu().numpy()
        labels = torch.argmax(labels, dim=-1).cpu().numpy()
        total += len(labels)
        correct += (pred == labels).sum().item()
        pbar.set_postfix(Loss=loss_train/(i+1), Accuracy=correct/total)

    loss_valid, acc_valid = calculate_loss_and_accuracy(model, criterion, val_loader, device, epoch, NUM_EPOCHS)
    log_valid.append([loss_valid, acc_valid])
    if best_acc < acc_valid:
      best_acc = acc_valid
      
      torch.save(model, os.path.join(ROOT, model_path))

  return log_valid

In [8]:
DROP_RATE = 0.1
OUTPUT_SIZE = 4
BATCH_SIZE = 16
NUM_EPOCHS = 10
LEARNING_RATE = 1e-6
VERSION = 'v5'

In [9]:
import sklearn.metrics
def train_val_predict(
        df_train,       # 学習用のデータ
        text_column,    # 対象のカラム名
        target_column,  # 目的変数のカラム名
        df_valid=None,  # 検証用データ
        df_val_test=None,       # 予測用データ
        model_file_prefix="",  # 保存時のファイル名識別子
        epochs=NUM_EPOCHS,
        batch_size=BATCH_SIZE,
    ):
  
    # model
    model_path = "{}_{}.pth".format(model_file_prefix, VERSION)
    model = BERTClass(DROP_RATE, OUTPUT_SIZE)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)
    device = 'cuda' if cuda.is_available() else 'cpu'

    # dataset
    df_val_test[target_column].fillna(0)

    max_len = 400
    tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
    dataset_train = CreateDataset(df_train[text_column].values, pd.get_dummies(df_train[target_column]).values, tokenizer, max_len)
    dataset_val = CreateDataset(df_valid[text_column].values, pd.get_dummies(df_valid[target_column]).values, tokenizer, max_len)
    dataset_test = CreateDataset(df_val_test[text_column].values, pd.get_dummies(df_val_test[target_column]).values, tokenizer, max_len)

    dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
    dataloader_valid = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=True)
    dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False)

    # train
    print('\n================  start train  ================')
    metric = train_val(dataloader_train, dataloader_valid, model_path)
    print('================  end train  ================\n')

    # predict
    model = torch.load(os.path.join(ROOT, model_path))
    model.eval()
    pred_y_list = []
    emb_list = []
    with torch.no_grad():
      with tqdm(dataloader_test) as pbar:
        pbar.set_description('test')
        for data in pbar:
          ids = data['ids'].to(device)
          mask = data['mask'].to(device)
          labels = data['labels'].to(device)

          output = model.forward(ids, mask)
          pred = torch.argmax(output, dim=-1).cpu().numpy()

          pred_y_list.extend(pred)
          emb_list.extend(output.cpu().numpy())
    return metric, pred_y_list, emb_list

In [10]:
import sklearn.model_selection

text_column, target_column, n_splits = ('description', 'jobflag', 3)

df = pd.concat([train, test], ignore_index=True, sort=False)
df_train = df[df[target_column].notnull()]
df_test = df[df[target_column].isnull()]

df_train_idx = df_train.index

# store result
df_pred = pd.DataFrame(df.index, columns=["index"]).set_index("index")
df_emb = pd.DataFrame(df.index, columns=["index"]).set_index("index")
df_emb_pred = None
metric_list = []
all_emb = []

# cross validation
kf = sklearn.model_selection.StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1234)
for i, (train_idx, test_idx) in enumerate(kf.split(df_train, df_train[target_column])):
  df_train_sub = df_train.iloc[train_idx]
  df_test_sub = df_train.iloc[test_idx]

  df_val_test = pd.concat([df_test_sub, df_test], ignore_index=True, sort=False)

  model_file_prefix = "cv_{}".format(i)

  # train
  metric, pred_y_list, emb_list = train_val_predict(
      df_train=df_train_sub, 
      text_column=text_column,
      target_column=target_column, 
      df_valid=df_test_sub,
      df_val_test=df_val_test,
      model_file_prefix=model_file_prefix,
  )
  metric_list.append(metric)
  all_emb.append(emb_list)
  
  # result_name = "result_{}".format(i)
  # df_pred.loc[df_train_idx[test_idx], result_name] = pred_y_list[:len(test_idx)]
  # df_pred.loc[df_test.index, result_name] = pred_y_list[len(test_idx):]

  # a = pd.DataFrame(emb_list[:len(test_idx)], index=df_train_idx[test_idx])
  # df_emb = df_emb.combine_first(a)

  # if df_emb_pred is None:
  #     df_emb_pred = pd.DataFrame(emb_list[len(test_idx):], index=df_test.index)
  # else:
  #     df_emb_pred += emb_list[len(test_idx):]


  # pred_y = df_pred.mean(axis=1)

  # df_emb_pred /= n_splits
  # df_emb = df_emb.combine_first(df_emb_pred)
  # break


================  start train  ================

epoch: 0


valid: 100%|██████████| 32/32 [00:07<00:00,  4.47it/s, Accuracy=0.332, Loss=1.35]



epoch: 1


valid: 100%|██████████| 32/32 [00:07<00:00,  4.49it/s, Accuracy=0.52, Loss=1.2]



epoch: 2


...

valid: 100%|██████████| 32/32 [00:07<00:00,  4.50it/s, Accuracy=0.703, Loss=0.765]



epoch: 9


valid: 100%|██████████| 32/32 [00:07<00:00,  4.50it/s, Accuracy=0.719, Loss=0.749]


================  end train  ================



test: 100%|██████████| 127/127 [00:27<00:00,  4.54it/s]


In [15]:
np.array(all_emb[0]).shape, np.array(all_emb[1]).shape, np.array(all_emb[2]).shape

((2023, 4), (2022, 4), (2022, 4))

In [19]:
all_emb_ = np.array([np.array(emb)[-len(df_test):] for emb in all_emb])

In [31]:
len(list(zip(*all_emb_)))

1517

In [37]:
df_emsemble = pd.DataFrame([[np.mean([v1, v2, v3]) for v1, v2, v3 in zip(emb1, emb2, emb3)] for emb1, emb2, emb3 in zip(*all_emb_)])
df_emsemble.to_csv(os.path.join(ROOT, 'emsembel_v5.csv'))

In [49]:
pred_emsesmble = pd.DataFrame([[id, np.argmax(arr)+1] for id, arr in zip(df_test.id, df_emsemble.to_numpy())])

In [52]:
pred_emsesmble.to_csv(os.path.join(ROOT, 'v5_esemble_pred.csv'), index=None, header=None)
# df_test

In [33]:
for emb1, emb2, emb3 in zip(*all_emb_):
  print(emb1, emb2, emb3)
  break

[ 2.271842   -0.66456294 -1.419578    0.16455367] [ 2.5085385  -0.62811625 -1.314392   -0.08950315] [ 2.2126133  -0.780459   -1.4644257   0.11359177]


検討事項

tqdmによるメモリ占有を解決(
[tqdmでメモリリークにハマった話（機械学習）](https://qiita.com/nanoseeing/items/72df19278377002cd881))